In [ ]:
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG ATGTGTAGCGA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG ATGTGTAGCGA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG GACCATCCCCT CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG CGTATCCCGAA CTGACGCGCACGAATTACAGATAGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AAACCGAAGGT CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG CGCCCAATCTA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCCTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AAACCGAAGGT CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA


#from slow R1

CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AATAGCACTGC CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG GAGATCTCAAA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG TGTTAAGGCCG CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AATAGCACTGC CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AATAGCACTGC CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG GAGATCTCAAA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG


In [7]:
#pd.read_csv("/corgi/otherdataset/ellenbushell/P28011/sampleinfo.txt",sep="\t")
#bc_file = pd.read_csv("/corgi/otherdataset/ellenbushell/P28011/bc.csv",sep=" ")


import gzip
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path
import regex as re

def open_gzip_optional(fname,mode="rt"):
    if str(fname).endswith("gz"):
        return gzip.open(fname,mode)
    elif str(fname).endswith("bz2"):
        return bz2.open(fname,mode)
    else:
        return open(fname,mode)
            
def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in dna[::-1]])

rootdir = Path("/corgi/otherdataset/ellenbushell/2023march_phit/")
#rootdir = Path("/corgi/otherdataset/ellenbushell/2023march_pools/")
fastqdir = rootdir / "fastq"

################ Read barcode collection
bc_file = pd.read_csv("/corgi/otherdataset/ellenbushell/barcode_to_gene_210920_FRANK.csv")
bc_file["mutant"]=[x.split("|")[0] for x in bc_file["gene"]]
bc_file["barcode"] = [x.upper() for x in bc_file["barcode"]]
dict_bc2mutant = dict(zip(bc_file["barcode"],bc_file["mutant"]))

################ Prepare parser
flank_regex = re.compile("GGCGG"+"(\w{8,16})"+"CTGAC")

################ Process all files
#sample_dict = {}
list_samples = []
list_counts = []
for seq_file in fastqdir.iterdir():
    if str(seq_file).endswith("fastq.gz"):
        
        #if len(list_samples)>5:
        #    break


        ####### Extract sample name
        #P28011_1172_S11_L001_R1_001.fastq.gz
        # => P28011_1172
        #sample = format_filename(seq_file)
        sample = seq_file.name.split("_S")[0]
        
        #Empty counts dictionary
        count_dict = dict(zip(list(bc_file["mutant"]), [0]*bc_file.shape[0]))
        count_dict["_other"] = 0 
        count_dict["_err"] = 0 
        
        #Reverse complement if read2
        do_revcomp = !("_R2_" in str(seq_file))
        
        if "_R1_" in seq_file.name:
            sample = sample+":R1"
        else:
            sample = sample+":R2"
        print(sample)
        
        
        #For all lines
        with open_gzip_optional(seq_file) as f1:
            it1 = iter(f1)
            count = 0
            while True:
                try:
                    
                    #Read one entry in FASTQ
                    next(it1)
                    theseq = next(it1).strip()
                    next(it1)
                    next(it1)
                    
                    if do_revcomp:
                        theseq = reverse_complement(theseq)

                    #print("> "+theseq)
                        
                    #Parse one entry
                    try:
                        
                        putative_barcode = re.search(flank_regex, theseq)[1]
                        #print(putative_barcode)
                        
                        putative_barcode = reverse_complement(putative_barcode)  #for this input file
                        
                        if putative_barcode in dict_bc2mutant.keys():
                            count_dict[dict_bc2mutant[putative_barcode]] += 1
                        else:
                            count_dict["_other"] += 1
                            #print("not in dict "+putative_barcode)

                    except TypeError:
                        count_dict["_err"] += 1
                        #print("type err: "+theseq)
                    
                    count=count+1

                except StopIteration:
                    break
        
        #Turn into an array, add to the list
        list_samples.append(sample)
        list_counts.append(
            np.array([count_dict["_other"], count_dict["_err"]]+[count_dict[i] for i in bc_file["mutant"]])
        )
        
        

### Turn it all into one matrix
df = pd.DataFrame(
    np.vstack(list_counts).T,
    columns = list_samples, 
    index = ["_other","_err"]+list(bc_file["mutant"]))

P28657_1001:R1
P28657_1001:R2
P28657_1002:R1
P28657_1002:R2
P28657_1003:R1
P28657_1003:R2
P28657_1004:R1
P28657_1004:R2
P28657_1005:R1
P28657_1005:R2
P28657_1006:R1
P28657_1006:R2
P28657_1007:R1
P28657_1007:R2
P28657_1008:R1
P28657_1008:R2
P28657_1009:R1
P28657_1009:R2
P28657_1010:R1
P28657_1010:R2
P28657_1011:R1
P28657_1011:R2
P28657_1012:R1
P28657_1012:R2
P28657_1013:R1
P28657_1013:R2
P28657_1014:R1
P28657_1014:R2
P28657_1015:R1
P28657_1015:R2
P28657_1016:R1
P28657_1016:R2


In [8]:
df

,P28657_1001:R1,P28657_1001:R2,P28657_1002:R1,P28657_1002:R2,P28657_1003:R1,P28657_1003:R2,P28657_1004:R1,P28657_1004:R2,P28657_1005:R1,P28657_1005:R2,...,P28657_1012:R1,P28657_1012:R2,P28657_1013:R1,P28657_1013:R2,P28657_1014:R1,P28657_1014:R2,P28657_1015:R1,P28657_1015:R2,P28657_1016:R1,P28657_1016:R2
_other,0,0,6,1,2,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
_err,408036,408036,255052,255057,637839,637841,330920,330920,352423,352423,...,628180,628180,222170,222168,228924,228924,231261,231260,273783,273783
PBANKA_093210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PBANKA_142090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PBANKA_031260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PBANKA_071030,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PBANKA_021280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PBANKA_051730,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PBANKA_010440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Check how well it worked
df[0:2]/df.sum(axis=0)




,P28657_1001:R1,P28657_1001:R2,P28657_1002:R1,P28657_1002:R2,P28657_1003:R1,P28657_1003:R2,P28657_1004:R1,P28657_1004:R2,P28657_1005:R1,P28657_1005:R2,...,P28657_1012:R1,P28657_1012:R2,P28657_1013:R1,P28657_1013:R2,P28657_1014:R1,P28657_1014:R2,P28657_1015:R1,P28657_1015:R2,P28657_1016:R1,P28657_1016:R2
_other,0.0,0.0,0.000024,0.000004,0.000003,0.0,0.0,0.0,0.000003,0.000003,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
_err,1.0,1.0,0.999976,0.999996,0.999997,1.0,1.0,1.0,0.999997,0.999997,...,1.0,1.0,1.0,0.999986,1.0,1.0,1.0,0.999996,1.0,1.0


In [6]:
df.to_csv(rootdir / "counts.csv")

